In [4]:
import requests
import json
import building_tokenizer
import shelve
from pprint import PrettyPrinter
pp = PrettyPrinter()
from collections import defaultdict, OrderedDict

In [5]:
sensor_dict = shelve.open('metadata/bacnet_devices.shelve')

In [6]:
propTagTypeDict = {'data_type': 'BACNet_DataType',
                'desc': "BACNet_Description",
                'jci_name': 'VendorGivenName',
                'unit': 'BACNet_Unit',
                'instance': 'BACNet_InstanceNumber',
                'type_str': 'BACNet_TypeString',
                'name': 'BACNet_Name'
               }
naeListDict = dict()
naeListDict['EBU3B'] = ['505', '506']
#naeListDict['Applied_Physics_and_Mathematics'] = ['514', '513', '604']
#naeListDict['Computer_Science_and_Engineering_Building'] = ['505', '506']
#naeListDict['Conrad_Prebys_Music_Center'] = ['523']
#naeListDict['San_Diego_Supercomputer_Center'] = ['668', '575', '524', '520']
#naeListDict['Literature_Building'] = ['553']
#naeListDict['Structural_and_Materials_Engineering_Building'] = ['572', '573', '574']
#naeListDict['Jacobs_Hall'] = ['620', '621', '622', '623', '650', '639', '638', '642', '643', '640', \
#                              '641', '637', '646', '644', '645']
#naeListDict['Warren_Lecture_Hall'] = ['555', '552']


In [7]:
baseurl = 'https://bd-testbed.ucsd.edu:81'
tagtypeurl = baseurl + '/api/tagtype'
with open("bdtestbed_secret.json", "r") as fp:
    cred = json.load(fp)
    cid = cred['cid']
    ckey = cred['ckey']
oauthurl = baseurl+"/oauth/access_token/client_id="+cid+"/client_secret="+ckey
resp = requests.get(oauthurl).json()
token = resp['access_token']
header = {"Authorization": "bearer " + token, 'content-type':'application/json'}
print token

7c00e6edd097a37170427469e721883a


In [8]:
i = 0

buildingName = 'Applied_Physics_and_Mathematics'
 #This is not used for building name but only for nae now.
                                                 #This is the contaminated building.   
naeList = naeListDict[buildingName]

#buildingName="TempBuilding"
buildingTagUrl = baseurl + '/api/building/%s/tags'%buildingName
addedTagValueSet = set()

sensorTagValueDict = defaultdict(dict)

propList = propTagTypeDict.keys()
tagTypeList = propTagTypeDict.values()

iterNum = -1
for naeNum in naeList:
    payload = dict()
    payload['data'] = {'name':'NAENum', 'value':naeNum}
    #resp = requests.post(buildingTagUrl, data=json.dumps(payload), headers=header)
    
    #print resp
    
    objList = sensor_dict[naeNum]['objs']
    for obj in objList[:iterNum]:
        tagTypeList = list()
        tagValueList = list()
        for prop, datum in obj.items():
            if prop=='props':
                for propProp, propDatum in datum.items():
                    if propProp in propList:
                        tagTypeList.append(propTagTypeDict[propProp])
                        tagValueList.append(str(propDatum))
            else:
                if prop in propList:
                    tagTypeList.append(propTagTypeDict[prop])
                    tagValueList.append(str(datum))
        assert(len(tagTypeList)==len(tagValueList))
        
        instNum = tagValueList[tagTypeList.index('BACNet_InstanceNumber')]
        datatype = tagValueList[tagTypeList.index('BACNet_DataType')]
        srcid = str(naeNum)+'_'+str(datatype)+'_'+str(instNum)
        if datatype not in ['0','1','2','3','4','5','13','14','19']:
            continue
        for tag, val in zip(tagTypeList, tagValueList):
            payload = dict()
            payload['data'] = {'name':tag, 'value':str(val)}
            i += 1
            if i%500==0:
                print i
            addedTagValueSet.add((tag,val))
            sensorTagValueDict[srcid][tag] = val
            resp = requests.post(buildingTagUrl, data=json.dumps(payload), headers=header)
        sensorTagValueDict[srcid]['NAENum'] = naeNum
            

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000


ConnectionError: HTTPSConnectionPool(host='mesl-bd.ucsd.edu', port=81): Max retries exceeded with url: /api/building/Applied_Physics_and_Mathematics/tags (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7fc7fa5676d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
len(sensorTagValueDict)

In [ ]:
# Create sensors
#buildingName="TempBuilding"
sensorUrl = baseurl + '/api/sensor'
for sensorName, tagValueDict in sensorTagValueDict.items():
    payload = defaultdict(dict)
    payload['data']['name'] = sensorName
    payload['data']['identifier'] = tagValueDict['NAENum']+'_'+tagValueDict['BACNet_DataType'] + '_' + tagValueDict['BACNet_InstanceNumber']
    payload['data']['building'] = buildingName
    payload = dict(payload)
    resp = requests.post(sensorUrl, data=json.dumps(payload), headers=header)
    uuid = resp.json()['uuid']
    tagValueDict['uuid'] = uuid

In [ ]:
# Adding tags to sensors
for sensorName, tagValueDict in sensorTagValueDict.items():
    sensorTagUrl = baseurl + '/api/sensor/'+tagValueDict['uuid']+'/tags'
    payload = dict()
    payload['data'] = list()
    for key, val in tagValueDict.items():
        payload['data'].append({'name': str(key), 'value':str(val)})
    payload = dict(payload)
    resp = requests.post(sensorTagUrl, data=json.dumps(payload), headers=header)

In [31]:
# Create Sensor Group
sensorGroupName = buildingName
sensorGroupUrl = baseurl + '/api/sensor_group'
data = {
    'name': sensorGroupName,
    'building': buildingName,
    'description': ''
}
resp = requests.post(sensorGroupUrl, data=json.dumps(data), headers=header)

In [32]:
# Create User Group
userGroupUrl = baseurl + '/api/user_group'
userGroupName = buildingName + '_Admin'
data = {
    'name': userGroupName,
    'description':''
}
resp = requests.post(userGroupUrl, data=json.dumps(data), headers=header)

In [34]:
# Adding tags to sensor groups
sensorGroupName = buildingName
sgTagsUrl = baseurl + '/api/sensor_group/%s/tags'%sensorGroupName
payload = dict()
payload['data'] = list()
for naeNum in naeList:
    payload['data'].append({'name': 'NAENum', 'value':str(naeNum)})
resp = requests.post(sgTagsUrl, data=json.dumps(payload), headers=header)
print resp.text

{
  "success": "True"
}



In [35]:
# Create Permission Pair
permissionUrl = baseurl + '/api/permission'
data = {
    'sensor_group':sensorGroupName,
    'user_group':userGroupName,
    'permission': 'rwp'
}
resp = requests.post(permissionUrl, data=json.dumps(data), headers=header)

In [36]:
resp.text

u'{\n  "error": "Missing data", \n  "success": "False"\n}\n'